In [1]:
# -*- coding:utf-8 -*-
import pandas as pd
import random
import fasttext
import jieba
from sklearn.model_selection import train_test_split
import re
from zhon.hanzi import punctuation

In [2]:
# 定义一级分类
farm = ['种植业','林业','渔业','农业综合','饲料','农药化肥']
coal = ['煤炭开采','石油开采','小金属','铜','特种钢','铅锌','黄金','铝','煤炭开采']
manu = ['食品','乳制品','啤酒','白酒','纺织','服饰','其他建材','家居用品','造纸','文教休闲','石油加工','化工原料','化学制药',
            '医疗保健','生物制药','中成药','化纤','日用化工','橡胶','塑料','玻璃','水泥','钢加工','矿物制品','元器件','机械基件',
            '农用机械','纺织机械','化工机械','轻工机械','机床制造','专用机械','工程机械','汽车整车','汽车配件','船舶','运输设备',
            '电气设备','电器连锁','家用电器','互联网','IT设备','通信设备','半导体','电器仪表','饲料','广告包装','陶瓷','普钢',
            '焦炭加工','红黄酒','软饮料','摩托车']
elec = ['供气供热','火力发电','新型电力','水力发电']
arch = ['建筑工程','装修装饰','染料涂料']
retail = ['批发业','超市连锁','商品城','百货']
trans = ['运输设备','路桥','公路','水运','港口','机场','航空','仓储物流','空运','铁路','公共交通']
acco = ['旅游服务','酒店餐饮','旅游景点']
info = ['通信设备','互联网','电信运营','软件服务']
eco = ['证券','多元金融','综合类','医药商业','其他商业','多元金融','石油贸易','保险']
house = ['房产服务','区域地产','全国地产','园区开发']
rent = ['商贸代理']
env = ['水务','环境保护']
service = ['汽车服务','汽车整车']
media = ['出版业','影视音像']
manage = ['银行']

In [3]:
def stop_process():
    stop = []
    standard_stop = []
    with open('./stopwords.txt','r',encoding='utf-8-sig') as f :
        lines = f.readlines()  # lines是list类型
        for line in lines:
            lline  = line.strip()     # line 是str类型,strip 去掉\n换行符
            stop.append(lline)        # 将stop 是列表形式

    for i in range(0,len(stop)):
        for word in stop[i].split():
            standard_stop.append(word)

    return standard_stop

In [4]:
# 一级分类选择
def cata1(label):
    if label in farm:
        return '农、林、牧、渔业',0
    elif label in coal:
        return '采矿业',1
    elif label in manu:
        return '制造业',2
    elif label in elec:
        return '电力、热力、燃气及水生产和供应业',3
    elif label in arch:
        return '建筑业',4
    elif label in retail:
        return '批发和零售业',5
    elif label in trans:
        return '交通运输、仓储和邮政业',6
    elif label in acco:
        return '住宿和餐饮业',7
    elif label in info:
        return '信息传输、软件和信息技术服务业',8
    elif label in eco:
        return '金融业',9
    elif label in house:
        return '房地产业',10
    elif label in rent:
        return '租赁和商务服务业',11
    elif label in env:
        return '水利、环境和公共设施管理业',12
    elif label in service:
        return '居民服务、修理和其他服务业',13
    elif label in media:
        return '文化、体育和娱乐业',14
    elif label in manage:
        return '公共管理、社会保障和社会组织',15
    
    return label+'??????????????'

In [5]:
# 返回第一次预测的下标
def get_index(label):
    # 有模型的大类
    if label == '农、林、牧、渔业':
        return 0
    elif label == '采矿业':
        return 1
    elif label == '制造业':
        return 2
    elif label == '电力、热力、燃气及水生产和供应业':
        return 3
    elif label == '建筑业':
        return 4
    elif label == '批发和零售业':
        return 5
    elif label == '交通运输、仓储和邮政业':
        return 6
    elif label == '住宿和餐饮业':
        return 7
    elif label == '信息传输、软件和信息技术服务业':
        return 8
    elif label == '金融业':
        return 9
    elif label == '房地产业':
        return 10
    elif label == '租赁和商务服务业':
        return 11
    elif label == '水利、环境和公共设施管理业':
        return 12
    elif label == '居民服务、修理和其他服务业':
        return 13
    elif label == '文化、体育和娱乐业':
        return 14
    elif label == '公共管理、社会保障和社会组织':
        return 15

In [6]:
# 数据预处理，用pandas读取csv文件中的特定数据
def pre_process(file):
    source = file
    df = pd.read_excel(source,encoding ="utf-8")
    df=df.dropna()                                                 #去掉空行
    scope = []
    scope1 = df['business_scope'].values.tolist()
    scope2 = df['main_business'].values.tolist()
    for s1,s2 in zip(scope1, scope2,): 
        seq = (s1,s2)
        s = "，".join(seq)
        scope.append(s)
    industry = df['industry'].values.tolist()
    #停用词
    stopwords = stop_process()
    
    #分句
    sentences=[]
    ori_sentences = []
    for m in range(16):
        ori_sentences.append([])
    for i,j in zip(scope,industry):
        #去除所有特殊符号
        string = re.sub('\W+', '', i).replace("_", '')
        string = re.sub(r"[0-9]", "", string)
        string = re.sub(r"[a-zA-Z]", "", string)
        
        segs=jieba.lcut(str(string))                               #jieba分词
        for i in range(len(segs)-1, -1, -1):
            if segs[i] in stopwords:
                del segs[i]
        #一级分类
        label,k = cata1(j)                
        sentences.append(" ".join(segs) + "\t" + "__label__" +str(label))
        #二级分类        
        ori_sentences[k].append(" ".join(segs) + "\t" + "__label__" + j)
    
    #乱序处理
    random.shuffle(sentences)
    
    #写入为txt
    out=open('./train_data.txt','w',encoding='utf-8')           
    for sentence in sentences:
        out.write(sentence+"\n")
        
    return ori_sentences


In [7]:
#载入训练数据集，进行数据预处理 -> 获得（训练集数据+__label__）
ori_sentences = pre_process('./沪交所.xlsx')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/r3/vcc58bx95zz8bl9cvt6twj2c0000gn/T/jieba.cache
Loading model cost 0.703 seconds.
Prefix dict has been built successfully.


In [53]:
#有监督学习训练数据集，导出模型为model_file.bin
classifier = fasttext.train_supervised(
    input = 'train_data.txt',
    label_prefix = '__label__',
    dim = 300,
    epoch = 50,
    lr = 1,
    lr_update_rate = 30,
    min_count = 1,
    loss = 'softmax',
    word_ngrams = 1,
    bucket = 2000000)
classifier.save_model("Model.bin")

In [54]:
#测试数据：一级分类划分 -> 获得
def test_model1(file):
    
    #预处理：获得正确的测试集参数
    #1. 训练数据预处理
    source = file
    df = pd.read_excel(source,encoding ="utf-8")
    df=df.dropna()                                                 #去掉空行
    # 合并 business_scope 和 main_business
    scope = []
    scope1 = df['business_scope'].values.tolist()
    scope2 = df['main_business'].values.tolist()
    for s1,s2 in zip(scope1, scope2,): 
        seq = (s1,s2)
        s = "，".join(seq)
        scope.append(s)
    industry=df['industry'].values.tolist()
    #停用词
    stopwords = stop_process()
    #分句
    sentences=[]
    test_sentences = []
    for n in range(16):
        test_sentences.append([])
    labels=[]
    test_labels = []
    for n in range(16):
        test_labels.append([])
    for i,j in zip(scope,industry):
        
        #2. labels=测试集正确的标签的一级分类
        J,k = cata1(j)                
        labels.append("".join(str(J)))         
        
        #3. 测试集sentences处理
        string = re.sub('\W+', '', i).replace("_", '')
        string = re.sub(r"[0-9]", "", string)
        string = re.sub(r"[a-zA-Z]", "", string)
        segs=jieba.lcut(str(string))                               #jieba分词
        for i in range(len(segs)-1, -1, -1):
            if segs[i] in stopwords:
                del segs[i]
        sentences.append(" ".join(segs))    #sentences=测试集句子
        tmp,k = cata1(j)                
        #4. test_sentences = 测试集句子+__label__+二级分类 
        #   test_labels =    测试集标签正确的二级分类        
        test_sentences[k].append(" ".join(segs) + "\t" + "__label__" + j)
        test_labels[k].append("".join(j))
    
    #5. 使用模型，获得预测结果
    out=open('./test_label.txt','w',encoding='utf-8')               #写入label和result对比文件
    classifier = fasttext.load_model('Model.bin') 
    total_counter = 0
    correct_counter = 0
    total_dict = {}
    correct_dict = {}
    
    #6. pre_sentences = 测试集句子+结果，按一级分类分 [[],[],[]...]
    pre_sentences = []
    for n in range(16):
        pre_sentences.append([])
    pre_labels = []
    for b in range(16):
        pre_labels.append([])
    
    for sentence,label in zip(sentences,labels):
        result = classifier.predict(sentence, k=1)
        
        for elem in result[0]:
            string = re.sub('__label__', '', elem).replace("_", '')
            
            if total_dict.get(string):                                  #有这个元素，total字典加1
                total_dict[string] = total_dict.get(string) + 1
            else:                                                 #没有这个元素，total字典新增且设为1，correct字典新增为0
                total_dict[string] = 1
                correct_dict[string] = 0
                
            total_counter += 1
            if label==string:
                correct_counter += 1
                correct_dict[string] = correct_dict.get(string) + 1
                out.write(label +" "+ string + "\n")                     #写入label和result对比文件
            else:
                out.write("??????"+label +" "+ string + "\n")                     #写入label和result对比文件
        
        index = get_index(string) 
        pre_sentences[index].append(" ".join(segs) + "\t" + "__label__" + label)   #
        pre_labels[index].append("".join(label))
            
    #计算全局精度        
    precision = (correct_counter / total_counter) * 100
    print('precision: {:.2f}%'.format(precision))
    print('')
    
    #统计二级分类各类准确率
    dict = {}
    output=open('分类精度统计.txt','w',encoding='utf-8')
    for i in total_dict.items():
        dict[i[0]] = correct_dict.get(i[0]) / total_dict.get(i[0]) * 100
        output.write(i[0] + ': {:.2f}%'.format(dict.get(i[0])) + "\t" +"(样本数量："+ str(total_dict.get(i[0])) + ")\n")
        
    return test_sentences, test_labels, pre_sentences, pre_labels
    
test_sentences,test_labels, pre_sentences, pre_labels = test_model1('./上交所.xlsx')

precision: 78.71%



In [55]:
#单一输入测试
classifier = fasttext.load_model('Model.bin')  
label = classifier.predict('证券 经纪 证券 投资 咨询 与 证券 交易 证券 投资 活动 有关 的 财务 顾问', k=1) 
for elem in label[0]:
    string = re.sub('__label__', '', elem).replace("_", '')
    print(string)

金融业


# 二级标题

In [56]:
# 二级分类训练模型，模型按照大类序号命名，其中，7、11、13样本不够没有模型

def process_2(ori_sentences,sentences,labels,model_name):
    #1. 写入为txt
    out=open('./subTrain/train_data.txt','w',encoding='utf-8')           
    for sentence in sentences:
        out.write(str(sentence)+"\n")
        
    #2. 训练模型
    classifier = fasttext.train_supervised(
    input = './subTrain/train_data.txt',
    label_prefix = '__label__',
    dim = 300,
    epoch = 50,
    lr = 1,
    lr_update_rate = 30,
    min_count = 1,
    loss = 'softmax',
    word_ngrams = 1,
    bucket = 2000000)
    classifier.save_model(model_name)
    
    #3. 使用模型预测结果
    classifier = fasttext.load_model(model_name)
    for senentence in sentences:
        result = classifier.predict(sentence, k=1)
        for elem in result[0]:
            string = re.sub('__label__', '', elem).replace("_", '')
    
    #4. 写入为txt
    out=open('./subTrain/test_label.txt','w',encoding='utf-8')               #写入label和result对比文件
    classifier = fasttext.load_model(model_name) 
    total_counter = 0
    correct_counter = 0
    total_dict = {}
    correct_dict = {}
    
    for sentence,label in zip(sentences,labels):
        result = classifier.predict(sentence, k=1)

        for elem in result[0]:
            string = re.sub('__label__', '', elem).replace("_", '')
            
            if total_dict.get(string):                                  #有这个元素，total字典加1
                total_dict[string] = total_dict.get(string) + 1
            else:                                                 #没有这个元素，total字典新增且设为1，correct字典新增为0
                total_dict[string] = 1
                correct_dict[string] = 0
                
            total_counter += 1
            if label==string:
                correct_counter += 1
                correct_dict[string] = correct_dict.get(string) + 1
            
            out.write(label +" "+ string + "\n")                     #写入label和result对比文件
            
    #计算全局精度        
    precision = (correct_counter / total_counter) * 100
    print('precision: {:.2f}%'.format(precision))
    print('')
    
    #统计二级分类各类准确率
    dict = {}
    output=open('./subTrain/分类精度统计.txt','a',encoding='utf-8')
    for i in total_dict.items():
        dict[i[0]] = correct_dict.get(i[0]) / total_dict.get(i[0]) * 100
        output.write(i[0] + ': {:.2f}%'.format(dict.get(i[0])) + "\t" +"(样本数量："+ str(total_dict.get(i[0])) + ")\n")
    return correct_counter, total_counter


# 执行第二次模型构建
all_correct = 0
all_total = 0
for index in range(16):
    if len(ori_sentences[index]) > 20 or len(test_sentences[index]) > 20:
        correct, total = process_2(ori_sentences[index],pre_sentences[index],pre_labels[index],"./subTrain/Model"+str(index)+".bin")
        all_correct += correct
        all_total += total
    else:
        print(str(index)+"- ????? 无法生成模型!")

# 计算所有精度
pre = all_correct / all_total * 100
print('****基于正确的一级分类****')
print('错误个数：' + str(all_total-all_correct))
print('precision: ' + ': {:.2f}%'.format(pre))

precision: 66.67%

precision: 80.33%



precision: 83.31%

precision: 74.00%



precision: 71.43%

precision: 53.06%



precision: 82.61%

7- ????? 无法生成模型!
precision: 57.14%



precision: 65.67%

precision: 73.77%

11- ????? 无法生成模型!


precision: 60.71%

13- ????? 无法生成模型!
precision: 86.96%

precision: 100.00%

****基于正确的一级分类****
错误个数：313
precision: : 78.69%


In [38]:
# 预测映射 1->2
def reflex(label):
    # 有模型的大类
    if label == '农、林、牧、渔业':
        return "./subTrain/Model0.bin"
    elif label == '采矿业':
        return "./subTrain/Model1.bin"
    elif label == '制造业':
        return "./subTrain/Model2.bin"
    elif label == '电力、热力、燃气及水生产和供应业':
        return "./subTrain/Model3.bin"
    elif label == '建筑业':
        return "./subTrain/Model4.bin"
    elif label == '批发和零售业':
        return "./subTrain/Model5.bin"
    elif label == '交通运输、仓储和邮政业':
        return "./subTrain/Model6.bin"
    elif label == '信息传输、软件和信息技术服务业':
        return "./subTrain/Model8.bin"
    elif label == '金融业':
        return "./subTrain/Model9.bin"
    elif label == '房地产业':
        return "./subTrain/Model10.bin"
    elif label == '水利、环境和公共设施管理业':
        return "./subTrain/Model12.bin"
    elif label == '文化、体育和娱乐业':
        return "./subTrain/Model14.bin"
    elif label == '公共管理、社会保障和社会组织':
        return "./subTrain/Model15.bin"
    
    #没有模型的大类（7，11，13）
    return 0

In [39]:
#单一输入测试(二级分类)
def predict(sample):
    classifier = fasttext.load_model('Model.bin')  
    label = classifier.predict(sample, k=1) 
    for elem in label[0]:
        string = re.sub('__label__', '', elem).replace("_", '')
        mode_addr = reflex(string)
        # 二级分类
        if mode_addr != 0:
            new_classifier = fasttext.load_model(mode_addr)  
            new_label = new_classifier.predict(sample, k=1)
            for ele in new_label[0]:
                new_string = re.sub('__label__', '', ele).replace("_", '')
                print(new_string)
        else:
            if string == '住宿和餐饮业':
                if ('服务' and '旅游' and '管理' and '开发') in sample:
                    print('旅游服务')
                    break
                elif ('酒店' and '餐饮') in sample:
                    print('酒店餐饮')
                    break
                elif ('景区' or '景点' or '房地产' or '客运' or '索道' or '建设') in sample:
                    print('旅游景点')
                    break
            elif string == '租赁和商务服务业':
                print('商贸代理')
                break
            elif string == '居民服务、修理和其他服务业':
                if '管理' in sample:
                    print('汽车服务')
                    break
                else:
                    print('汽车整车')
                    break
            else:
                print('其他')
            

predict('证券 经纪 证券 投资 咨询 与 证券 交易 证券 投资 活动 有关 的 财务 顾问')
predict('酒店 餐饮 客房 产品')

金融业
酒店餐饮
